### obtaining the movie review dataset

#### http://ai.stanford.edu/~amaas/data/sentiment/

In [ ]:
import pandas as pd
import os

In [ ]:
basepath = 'C:/sentiments/aclImdb/'

In [ ]:
labels = {'pos':1,'neg':0}

In [ ]:
df = pd.DataFrame()

In [ ]:
for s in ('test','train'):
    for l in ('pos','neg'):
        path = os.path.join(basepath,s,l)
        
        for fil in os.listdir(path):
           
            
            with open(os.path.join(path,fil),'rb') as f:
                txt = f.read()
            df = df.append([[txt,labels[l]]],ignore_index=True)
            
df.columns=['review','sentiment']

In [ ]:
df.head()

In [ ]:
df.sentiment.value_counts()

In [ ]:
import numpy as np

In [ ]:
from sklearn.utils import shuffle

In [ ]:
df = shuffle(df)

In [ ]:
df.head()

In [ ]:
df.to_csv('movie_reviews.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np
#from nltk.corpus import stopwords

In [ ]:
#stop = stopwords.words('english')

In [ ]:

df_new = pd.read_csv('movie_reviews.csv')

In [ ]:
from sklearn.utils import shuffle
df_new = shuffle(df_new)

In [ ]:
df_new.head()

In [ ]:
df_new.sentiment.value_counts()

In [ ]:
#df_new.loc[10,'review']
#[df_new.review.str.contains('<br /><br />')]

\W =matches any non-alphanumeric character;

\D = matches any non-digit character

<[^>]*> :all tag  ex..<br />, <a>
    
[^>] :except '>'

In [ ]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)',text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

# \W =matches any non-alphanumeric character;
# \D = matches any non-digit character
#<[^>]*> :all tag  ex..<br />, <a>
# [^>] :except '>' 

In [ ]:
preprocessor("</a>This is# a ;test ::-)!</a>")

In [ ]:
df_new['review'] = df_new['review'].apply(preprocessor)

In [ ]:
df_new.head()
#df_new[df_new.review.str.contains('<br /><br />')]

In [ ]:
df_new.index=range(50000)

In [ ]:
df_new.head()

In [ ]:
X_train = df_new.loc[:2500, 'review'].values
y_train = df_new.loc[:2500, 'sentiment'].values
X_test = df_new.loc[2500:5000, 'review'].values
y_test = df_new.loc[2500:5000, 'sentiment'].values

In [ ]:
print ( np.bincount(y_test))
print (np.unique(y_test))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import GridSearchCV     #hyper-parameter tunning
from sklearn.model_selection import cross_val_score  #avg testing score

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
# X_new=tfidf.fit_transform(X_train)

#log =LogisticRegression()
#log.fit(X_new,y_train)
param_grid = {'clf__C': [1.0, 10.0, 100.0]}

In [ ]:
X_train.ndim

In [ ]:
lr_tfidf = Pipeline([('vect', tfidf),
                     
                     ('clf', LogisticRegression())])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy')

#lr_tfidf.steps

In [ ]:
gs_lr_tfidf.fit(X_train, y_train)
#print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
#print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

In [ ]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

In [ ]:
clf = gs_lr_tfidf.best_estimator_

In [ ]:
clf.score(X_test,y_test)

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nb = Pipeline([('vect', tfidf),
               ('clf', MultinomialNB())])


In [ ]:
nb.fit(X_train,y_train)

In [ ]:
nb.score(X_train,y_train)

In [ ]:
nb.score(X_test,y_test)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [ ]:
#cv = CountVectorizer(stop_words='english')
cv = TfidfVectorizer(stop_words='english')

In [ ]:
new_data = cv.fit_transform(X_train)
new_test = cv.transform(X_test)

In [ ]:
new_data.shape

In [ ]:
nb = MultinomialNB()
nb.fit(new_data,y_train)

In [ ]:
nb.score(new_test,y_test)

In [ ]:
nb.score(new_data,y_train)